In [34]:
import numpy as np
import pandas as pd
import os

from scipy import stats

os.chdir('/home/galina/PycharmProjects/Peptide Signal')

# Helper function, finds weight of aminoacid

In [35]:
def weight(name, dictionary):
    w = 0
    for e in name:
        w = w + dictionary[e]
    return int(w)

# Aminoacid weights to dictionary

In [36]:
amino = pd.read_csv('Aminoacids.csv', header = 0)
amino.head()

,Aminoacid,1-Letter Code,"Molecular weight, kDa"
0,Alanine,A,89
1,Arginine,R,174
2,Asparagine,N,132
3,Aspartic,D,133
4,Cysteine,C,121


In [37]:
amino.drop('Aminoacid', axis = 1, inplace = True)
amino.columns = ['Code', 'Weight']
weights = amino.set_index('Code')['Weight'].to_dict()
weights

{nan: nan,
 'A': 89.0,
 'C': 121.0,
 'D': 133.0,
 'E': 146.0,
 'F': 165.0,
 'G': 75.0,
 'H': 155.0,
 'I': 131.0,
 'K': 146.0,
 'L': 131.0,
 'M': 149.0,
 'N': 132.0,
 'P': 115.0,
 'Q': 147.0,
 'R': 174.0,
 'S': 105.0,
 'T': 119.0,
 'V': 117.0,
 'W': 204.0,
 'Y': 181.0}

# Peptide data cleaning

In [38]:
data = pd.read_csv('Peptides.csv', header = 0)

data.drop(['Grid Index',
           'Peptide Signal Uniformity',
           'Streptavidin Signal Uniformity',
           
           'Streptavidin Association Signal (Saturated)',
           'Streptavidin Dissociation Signal (Saturated)',
           
           'Tag',
           'Alignment',
           'Empty',
           'Thrombin',
           'PSA',
           'StrepTag', 
           'Myc', 
           'FLAG',
           'NA', 
           'Epitope', 
           'Bad Washed',
           'Bad Strep',
           'aMyc Signal Uniformity'], axis = 1, inplace = True)

# remove data with 'Bad Washed' flag
# data = data[ data['Bad Washed'] == 0 ]
# data = data[ data['Bad Strep'] == 0 ]

# data.drop(['Bad Washed', 'Bad Strep'], axis = 1, inplace = True)

data.head()

,Peptide,Peptide Length,Peptide Signal,Streptavidin Signal,aMyc Association Signal,aMyc Dissociation Signal,aMyc Signal
0,WTHPQFAT,8,0.000642,0.001180,0.000086,0.000006,-0.000015
1,WTHPQFAT,8,0.001130,0.001210,-0.000195,-0.000060,-0.000013
2,WTHLQFAT,8,0.000244,0.000022,0.000019,-0.000035,0.000058
3,NGQFQVWIPLAQK,13,0.002490,-0.000024,0.000547,0.000392,0.000177
4,LQLHPQAGK,9,0.003110,0.000408,0.000086,0.000095,0.000104


In [39]:
# remove NaNs and Xs
data = data.dropna()
data = data[ data['Peptide'] != 'X' ]

data.sort('Peptide', inplace = True)
data = data.reset_index(drop = True)

data.head()

,Peptide,Peptide Length,Peptide Signal,Streptavidin Signal,aMyc Association Signal,aMyc Dissociation Signal,aMyc Signal
0,AGQFQVWIPGAQK,13,0.00253,0.000011,0.000125,0.000068,0.000092
1,AGQFQVWIPGAQK,13,0.00279,-0.000060,0.000130,0.000026,0.000054
2,AGQFQVWIPGAQK,13,0.00336,0.001040,-0.000087,0.000084,-0.000019
3,AGQFQVWIPGAQK,13,0.00255,-0.000037,-0.000057,0.000032,0.000068
4,AGQFQVWIPGAQK,13,0.00187,0.000004,0.000182,0.000182,0.000046


# Calculate mean and standard deviation by peptide

In [40]:
grouped = data.groupby(['Peptide',
                        'Peptide Length'])                       

f = {'Peptide Signal'           : [np.mean, np.std, stats.sem, len],
     'Streptavidin Signal'      : [np.mean, np.std, stats.sem],
     'aMyc Signal'              : [np.mean, np.std, stats.sem],
     'aMyc Association Signal'  : [np.mean, np.std, stats.sem],
     'aMyc Dissociation Signal' : [np.mean, np.std, stats.sem]}     

In [41]:
result = grouped.agg(f)

# result.columns = result.columns.droplevel()
result.reset_index(inplace = True)

result.columns = ['Peptide',
                  'Peptide Length',    
                  
                  'Streptavidin Signal Mean',
                  'Streptavidin Signal SD',
                  'Streptavidin Signal SE',

                  'aMyc Dissociation Signal Mean',
                  'aMyc Dissociation Signal SD',
                  'aMyc Dissociation Signal SE',

                  'aMyc Association Signal Mean',
                  'aMyc Association Signal SD',
                  'aMyc Association Signal SE', 
                    
                  'aMyc Signal Mean',
                  'aMyc Signal SD',
                  'aMyc Signal SE',
                   
                  'Peptide Signal Mean',
                  'Peptide Signal SD',
                  'Peptide Signal SE',
                  
                  'Number of Spots']


result['Streptavidin Signal Mean NN'] = abs( result['Streptavidin Signal Mean']
                                          * (result['Streptavidin Signal Mean'] > 0) )
result['Peptide Signal Mean NN'] = abs( result['Peptide Signal Mean']
                                     * (result['Peptide Signal Mean'] > 0) )
result['aMyc Signal Mean NN'] = abs( result['aMyc Signal Mean']
                                  * (result['aMyc Signal Mean'] > 0) )
result['aMyc Association Signal Mean NN'] = abs( result['aMyc Association Signal Mean']
                                              * (result['aMyc Association Signal Mean'] > 0) )
result['aMyc Dissociation Signal Mean NN'] = abs( result['aMyc Dissociation Signal Mean']
                                               * (result['aMyc Dissociation Signal Mean'] > 0) )

result.head()

,Peptide,Peptide Length,Streptavidin Signal Mean,Streptavidin Signal SD,Streptavidin Signal SE,aMyc Dissociation Signal Mean,aMyc Dissociation Signal SD,aMyc Dissociation Signal SE,aMyc Association Signal Mean,aMyc Association Signal SD,...,aMyc Signal SE,Peptide Signal Mean,Peptide Signal SD,Peptide Signal SE,Number of Spots,Streptavidin Signal Mean NN,Peptide Signal Mean NN,aMyc Signal Mean NN,aMyc Association Signal Mean NN,aMyc Dissociation Signal Mean NN
0,AGQFQVWIPGAQK,13,0.000125,0.000307,0.000069,0.000129,0.000078,0.000017,0.000141,0.000125,...,0.000013,0.002625,0.000342,0.000077,20,0.000125,0.002625,7.802500e-05,0.000141,0.000129
1,AISPPPK,7,0.000083,0.000252,0.000025,0.000209,0.000517,0.000052,0.000252,0.000673,...,0.000043,0.002837,0.000502,0.000050,100,0.000083,0.002837,1.593498e-04,0.000252,0.000209
2,AKFPIPLGKQSG,12,0.000400,0.000516,0.000258,0.000134,0.000120,0.000060,0.000143,0.000133,...,0.000044,0.002910,0.000384,0.000192,4,0.000400,0.002910,7.076637e-05,0.000143,0.000134
3,AQWHPQAGK,9,0.000591,0.000707,0.000353,0.000041,0.000129,0.000064,0.000070,0.000144,...,0.000040,0.002933,0.000310,0.000155,4,0.000591,0.002933,1.000000e-07,0.000070,0.000041
4,ATHPQFAT,8,0.000264,0.000489,0.000244,0.000889,0.001586,0.000793,0.001080,0.001879,...,0.000773,0.001607,0.000362,0.000181,4,0.000264,0.001607,8.146400e-04,0.001080,0.000889


# Add weight, and sort

In [42]:
result['Peptide Weight'] = result['Peptide'].map(lambda x: weight(x, weights))
result.sort(['Peptide Length', 'Peptide'], inplace = True)
result = result.reset_index(drop = True)

result.head()

,Peptide,Peptide Length,Streptavidin Signal Mean,Streptavidin Signal SD,Streptavidin Signal SE,aMyc Dissociation Signal Mean,aMyc Dissociation Signal SD,aMyc Dissociation Signal SE,aMyc Association Signal Mean,aMyc Association Signal SD,...,Peptide Signal Mean,Peptide Signal SD,Peptide Signal SE,Number of Spots,Streptavidin Signal Mean NN,Peptide Signal Mean NN,aMyc Signal Mean NN,aMyc Association Signal Mean NN,aMyc Dissociation Signal Mean NN,Peptide Weight
0,AISPPPK,7,0.000083,0.000252,0.000025,0.000209,0.000517,0.000052,0.000252,0.000673,...,0.002837,0.000502,0.000050,100,0.000083,0.002837,0.000159,0.000252,0.000209,816
1,THPQFAT,7,-0.000017,0.000034,0.000020,0.000111,0.000098,0.000057,0.000113,0.000117,...,0.001647,0.000257,0.000148,3,0.000000,0.001647,0.000121,0.000113,0.000111,909
2,WHPQFAT,7,0.000526,0.000605,0.000302,0.000074,0.000121,0.000060,0.000110,0.000149,...,0.000924,0.000134,0.000067,4,0.000526,0.000924,0.000029,0.000110,0.000074,994
3,WTHPFAT,7,0.000311,0.000588,0.000294,0.000053,0.000077,0.000039,0.000047,0.000207,...,0.001179,0.000203,0.000101,4,0.000311,0.001179,0.000104,0.000047,0.000053,966
4,WTHPQAT,7,0.000049,0.000054,0.000031,0.000118,0.000188,0.000108,0.000221,0.000179,...,0.001800,0.000416,0.000240,3,0.000049,0.001800,0.000108,0.000221,0.000118,948


In [43]:
# rearrange order of columns
result = result.reindex_axis(['Peptide',
                              'Number of Spots',
                              'Peptide Length',
                              'Peptide Weight',
                              
                              'Peptide Signal Mean',
                              'Peptide Signal Mean NN',
                              'Peptide Signal SD',
                              'Peptide Signal SE',
                              
                              'Streptavidin Signal Mean',
                              'Streptavidin Signal Mean NN',
                              'Streptavidin Signal SD',
                              'Streptavidin Signal SE',
                              
                              'aMyc Signal Mean',
                              'aMyc Signal Mean NN',
                              'aMyc Signal SD',
                              'aMyc Signal SE',              
                              
                              'aMyc Dissociation Signal Mean',
                              'aMyc Dissociation Signal Mean NN',
                              'aMyc Dissociation Signal SD',
                              'aMyc Dissociation Signal SE',

                              'aMyc Association Signal Mean',
                              'aMyc Association Signal Mean NN',
                              'aMyc Association Signal SD',
                              'aMyc Association Signal SE'], axis=1)
result.head()

,Peptide,Number of Spots,Peptide Length,Peptide Weight,Peptide Signal Mean,Peptide Signal Mean NN,Peptide Signal SD,Peptide Signal SE,Streptavidin Signal Mean,Streptavidin Signal Mean NN,...,aMyc Signal SD,aMyc Signal SE,aMyc Dissociation Signal Mean,aMyc Dissociation Signal Mean NN,aMyc Dissociation Signal SD,aMyc Dissociation Signal SE,aMyc Association Signal Mean,aMyc Association Signal Mean NN,aMyc Association Signal SD,aMyc Association Signal SE
0,AISPPPK,100,7,816,0.002837,0.002837,0.000502,0.000050,0.000083,0.000083,...,0.000430,0.000043,0.000209,0.000209,0.000517,0.000052,0.000252,0.000252,0.000673,0.000067
1,THPQFAT,3,7,909,0.001647,0.001647,0.000257,0.000148,-0.000017,0.000000,...,0.000059,0.000034,0.000111,0.000111,0.000098,0.000057,0.000113,0.000113,0.000117,0.000068
2,WHPQFAT,4,7,994,0.000924,0.000924,0.000134,0.000067,0.000526,0.000526,...,0.000036,0.000018,0.000074,0.000074,0.000121,0.000060,0.000110,0.000110,0.000149,0.000074
3,WTHPFAT,4,7,966,0.001179,0.001179,0.000203,0.000101,0.000311,0.000311,...,0.000047,0.000023,0.000053,0.000053,0.000077,0.000039,0.000047,0.000047,0.000207,0.000103
4,WTHPQAT,3,7,948,0.001800,0.001800,0.000416,0.000240,0.000049,0.000049,...,0.000037,0.000021,0.000118,0.000118,0.000188,0.000108,0.000221,0.000221,0.000179,0.000103


In [44]:
result.to_csv('result.csv', index = False)